<a href="https://colab.research.google.com/github/sedrasattout/URL/blob/main/URLs_Graduation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install catboost
!pip install tldextract
!pip install xgboost
!pip install tensorflow.keras
!pip install requests
!pip install python-whois
# ================ استيراد المكتبات ================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
import re
import time
import warnings
import ipaddress
import random
from concurrent.futures import ThreadPoolExecutor
warnings.filterwarnings('ignore')
# ================ مكتبات التعلم الآلي ================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report, roc_curve, auc
# ================ مكتبات التعلم العميق ================
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping
    TENSORFLOW_AVAILABLE = True
except ImportError:
    TENSORFLOW_AVAILABLE = False
# ================ مكتبات إضافية ================
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
try:
    from catboost import CatBoostClassifier
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False
# ================ إعدادات الداتا سيت ================
# الداتا سيت الأولى (للنظام الكامل)
MAIN_DATA_FILE = "/content/balanced_urls.csv"
# الداتا سيت الثانية (للخوارزمية فقط)
RESERVED_ONLY_DATA_FILE = "/content/reserved_ips_dataset.csv"

class UnifiedModelSelector:
    """محدد نموذج موحد لاختيار أفضل نموذج"""

    def __init__(self, min_requirements=None):
        if min_requirements is None:
            self.min_requirements = {
                'min_auc': 0.7,
                'min_recall': 0.8,
                'min_accuracy': 0.7,
                'min_f1': 0.7,
                'min_precision': 0.6
            }
        else:
            self.min_requirements = min_requirements

        self.metric_weights = {
            'auc': 0.25,
            'recall': 0.25,
            'f1': 0.20,
            'precision': 0.15,
            'accuracy': 0.15
        }

    def choose_pareto_optimal(self, models_results):
        """اختيار أفضل نموذج"""
        print("\n" + "="*100)
        print("🏆 اختيار أفضل نموذج باستخدام Pareto ")

        # تحويل النتائج
        models_list = []
        for name, metrics in models_results.items():
            models_list.append({
                'name': name,
                'model': metrics.get('model'),
                'accuracy': metrics.get('accuracy', 0),
                'precision': metrics.get('precision', 0),
                'recall': metrics.get('recall', 0),
                'f1': metrics.get('f1_score', 0),
                'auc': metrics.get('auc_roc', 0),
                'specificity': metrics.get('specificity', 0),
                'training_time': metrics.get('training_time', 0)
            })

        # تطبيق المتطلبات
        eligible_models = []
        for model in models_list:
            if all([
                model['auc'] >= self.min_requirements.get('min_auc', 0),
                model['recall'] >= self.min_requirements.get('min_recall', 0),
                model['accuracy'] >= self.min_requirements.get('min_accuracy', 0),
                model['f1'] >= self.min_requirements.get('min_f1', 0),
                model['precision'] >= self.min_requirements.get('min_precision', 0)
            ]):
                eligible_models.append(model)

        if not eligible_models:
            print("❌ لا يوجد نموذج يلبي المتطلبات!")
            return None, None

        # Pareto Front
        pareto_front = []
        for i, model_i in enumerate(eligible_models):
            dominated = False
            for j, model_j in enumerate(eligible_models):
                if i != j and self._dominates(model_j, model_i):
                    dominated = True
                    break
            if not dominated:
                pareto_front.append(model_i)

        # اختيار النهائي
        if len(pareto_front) > 1:
            for model in pareto_front:
                model['weighted_score'] = sum(
                    model[metric] * weight
                    for metric, weight in self.metric_weights.items()
                )
            best_model = max(pareto_front, key=lambda x: x['weighted_score'])
        else:
            best_model = pareto_front[0]

        # عرض النتائج
        print(f"\n✅ النموذج المختار: {best_model['name']}")
        print(f"📊 Recall: {best_model['recall']:.4f} | AUC: {best_model['auc']:.4f} | F1: {best_model['f1']:.4f}")

        return best_model['name'], {'selected_model': best_model}

    def _dominates(self, a, b):
        """تحقق إذا كان a يهيمن على b"""
        metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
        all_better = all(a[m] >= b[m] for m in metrics)
        some_better = any(a[m] > b[m] for m in metrics)
        return all_better and some_better

class OptimizedURLFeatureExtractor:
    """فئة محسنة لاستخراج الميزات مع تحسين السرعة"""
    def __init__(self):
        # تحميل الأنماط مسبقاً لتحسين السرعة
        self.ipv4_pattern = re.compile(r'^(\d{1,3}\.){3}\d{1,3}(:\d+)?$')
        self.ipv6_pattern = re.compile(r'^[0-9a-fA-F:]+$')
        self.digit_pattern = re.compile(r'\d')
        self.special_char_pattern = re.compile(r'[^a-zA-Z0-9]')

        # تحضير القوائم كمجموعات للبحث السريع
        self.suspicious_keywords = set([
            'login', 'secure', 'account', 'banking', 'verify', 'confirm',
            'update', 'password', 'credential', 'billing', 'payment', 'admin',
            'portal', 'dashboard', 'config', 'control', 'server', 'authenticate',
            'validation', 'signin', 'authorize', 'transaction', 'transfer'
        ])

        self.benign_keywords = set([
            'info', 'status', 'help', 'docs', 'documentation', 'support',
            'about', 'contact', 'test', 'health', 'api', 'public', 'open'
        ])

        self.high_risk_keywords = set(['admin', 'login', 'password', 'banking', 'payment', 'verify'])
        self.short_domains = set(['bit.ly', 'tinyurl.com', 'goo.gl', 't.co', 'ow.ly'])
        self.suspicious_tlds = set(['tk', 'ml', 'ga', 'cf', 'xyz', 'top', 'club', 'loan'])

        # ذاكرة تخزين مؤقت للنتائج
        self._feature_cache = {}

    def _extract_fast_ip_features(self, hostname):
        """استخراج ميزات IP بسرعة"""
        features = {
            'is_ip_address': 0, 'is_private_ip': 0, 'is_public_ip': 0,
            'is_localhost': 0, 'is_multicast': 0, 'is_link_local': 0,
            'is_loopback': 0, 'is_reserved_ip': 0, 'is_broadcast': 0,
            'is_documentation': 0, 'ip_category': 0, 'ip_class': 0
        }

        try:
            # التحقق إذا كان hostname هو عنوان IP
            if self.ipv4_pattern.match(hostname) or self.ipv6_pattern.match(hostname):
                features['is_ip_address'] = 1
                ip = ipaddress.ip_address(hostname.split(':')[0])

                features['is_private_ip'] = 1 if ip.is_private else 0
                features['is_public_ip'] = 1 if not ip.is_private and not ip.is_reserved else 0
                features['is_localhost'] = 1 if ip.is_loopback else 0
                features['is_multicast'] = 1 if ip.is_multicast else 0
                features['is_link_local'] = 1 if ip.is_link_local else 0
                features['is_loopback'] = 1 if ip.is_loopback else 0
                features['is_reserved_ip'] = 1 if ip.is_reserved else 0

                # فئة IP
                if isinstance(ip, ipaddress.IPv4Address):
                    first_byte = int(str(ip).split('.')[0])
                    if first_byte <= 127:
                        features['ip_class'] = 1  # Class A
                    elif first_byte <= 191:
                        features['ip_class'] = 2  # Class B
                    elif first_byte <= 223:
                        features['ip_class'] = 3  # Class C
                    else:
                        features['ip_class'] = 4  # Class D/E
        except:
            pass

        return features

    def extract_features_optimized(self, url):
        """استخراج الميزات بشكل محسن للسرعة"""
        # التحقق من الذاكرة المؤقتة أولاً
        if url in self._feature_cache:
            return self._feature_cache[url]

        features = {}
        try:
            parsed = urlparse(url)
            url_lower = url.lower()
            hostname = parsed.netloc

            # الميزات الأساسية السريعة
            features['url_length'] = len(url)
            features['host_length'] = len(hostname)
            features['path_length'] = len(parsed.path)
            features['num_digits'] = len(self.digit_pattern.findall(url))
            features['num_special_chars'] = len(self.special_char_pattern.findall(url))
            features['num_dots'] = url.count('.')
            features['num_dashes'] = url.count('-')
            features['num_underscores'] = url.count('_')
            features['has_https'] = 1 if parsed.scheme == 'https' else 0
            features['num_parameters'] = len(parsed.query.split('&')) if parsed.query else 0
            features['num_subdirectories'] = parsed.path.count('/') - 1 if parsed.path else 0

            # البحث السريع باستخدام المجموعات
            features['suspicious_keywords_count'] = sum(1 for keyword in self.suspicious_keywords if keyword in url_lower)
            features['benign_keywords_count'] = sum(1 for keyword in self.benign_keywords if keyword in url_lower)
            features['keywords_balance'] = features['suspicious_keywords_count'] - features['benign_keywords_count']
            features['high_risk_keywords'] = sum(1 for keyword in self.high_risk_keywords if keyword in url_lower)
            features['is_shortened'] = 1 if any(domain in url_lower for domain in self.short_domains) else 0

            # استخراج TLD سريع
            netloc_parts = hostname.split('.')
            tld = netloc_parts[-1] if len(netloc_parts) >= 2 else 'unknown'
            features['tld'] = tld
            features['is_suspicious_tld'] = 1 if tld in self.suspicious_tlds else 0

            # ميزات الـ IP السريعة
            ip_features = self._extract_fast_ip_features(hostname)
            features.update(ip_features)

            # ميزات إضافية سريعة
            features['has_port'] = 1 if ':' in hostname else 0
            features['has_at_symbol'] = 1 if '@' in url else 0
            features['has_redirect'] = 1 if 'redirect' in url_lower or 'url=' in url_lower else 0

        except Exception:
            # استخدام القيم الافتراضية بسرعة
            features = self._get_default_features()

        # تخزين في الذاكرة المؤقتة
        self._feature_cache[url] = features
        return features

    def _get_default_features(self):
        """إرجاع الميزات الافتراضية بسرعة"""
        return {
            'url_length': 0, 'host_length': 0, 'path_length': 0, 'num_digits': 0,
            'num_special_chars': 0, 'num_dots': 0, 'num_dashes': 0, 'num_underscores': 0,
            'has_https': 0, 'num_parameters': 0, 'num_subdirectories': 0,
            'suspicious_keywords_count': 0, 'benign_keywords_count': 0, 'keywords_balance': 0,
            'high_risk_keywords': 0, 'is_shortened': 0, 'tld': 'unknown', 'is_suspicious_tld': 0,
            'has_port': 0, 'has_at_symbol': 0, 'has_redirect': 0,
            'is_ip_address': 0, 'is_private_ip': 0, 'is_public_ip': 0, 'is_localhost': 0,
            'is_multicast': 0, 'is_link_local': 0, 'is_loopback': 0, 'is_reserved_ip': 0,
            'is_broadcast': 0, 'is_documentation': 0, 'ip_category': 0, 'ip_class': 0
        }

    def extract_features_batch(self, urls):
        """استخراج الميزات لمجموعة من الروابط باستخدام التزامن"""
        print(f"🔍 استخراج الميزات لـ {len(urls)} رابط...")

        # استخدام التزامن لتحسين الأداء
        with ThreadPoolExecutor(max_workers=4) as executor:
            features_list = list(executor.map(self.extract_features_optimized, urls))

        features_df = pd.DataFrame(features_list)

        # معالجة TLD بسرعة
        if 'tld' in features_df.columns:
            features_df['tld_encoded'] = features_df['tld'].astype('category').cat.codes
            features_df = features_df.drop('tld', axis=1)

        # تنظيف البيانات بسرعة
        features_df = features_df.fillna(0).replace([np.inf, -np.inf], 0)

        return features_df

class OptimizedDataProcessor:
    """معالج بيانات محسن للسرعة"""

    def __init__(self):
        self.feature_extractor = OptimizedURLFeatureExtractor()

    def prepare_data_fast(self, data):

        # استخراج الميزات بشكل جماعي
        features_df = self.feature_extractor.extract_features_batch(data['url'])

        # دمج سريع
        final_data = pd.concat([data.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
        final_data = final_data.drop_duplicates()

        # إزالة الأعمدة غير الضرورية بسرعة
        cols_to_drop = ['url', 'label'] if 'label' in final_data.columns else ['url']
        feature_cols = [col for col in final_data.columns if col not in cols_to_drop + ['result']]

        print(f"✅ تم تحضير {len(final_data)} سجل مع {len(feature_cols)} ميزة")
        return final_data, feature_cols

class Top7ModelsClassifier:
    """مصنف بأفضل 7 نماذج فقط"""

    def __init__(self):
        self.data_processor = OptimizedDataProcessor()
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_accuracy = 0
        self.pareto_selector = UnifiedModelSelector()
    def select_best_with_pareto(self):
        """اختيار أفضل نموذج باستخدام Pareto Optimal"""

        if not self.results:
            print("❌ لا توجد نتائج للاختيار منها!")
            return None
        # استخدام الـ Pareto selector
        best_name, details = self.pareto_selector.choose_pareto_optimal(self.results)

        if best_name:
            # تحديث أفضل نموذج
            self.best_model = self.results[best_name]['model']
            self.best_accuracy = self.results[best_name]['accuracy']

        return best_name, details

    def load_data(self, file_path=None):
        """تحميل البيانات"""
        print(" تحميل البيانات...")
        self.raw_data = self.data_processor.load_and_validate_data(file_path)
        self.processed_data, self.feature_names = self.data_processor.prepare_data_fast(self.raw_data)
        return self.processed_data

    def split_data(self, test_size=0.3):
        """تقسيم البيانات"""
        if 'result' not in self.processed_data.columns:
            raise ValueError("البيانات لا تحتوي على عمود 'result'")

        feature_cols = [col for col in self.processed_data.columns if col not in ['url', 'result', 'label']]
        X = self.processed_data[feature_cols]
        y = self.processed_data['result']

        print(f" شكل البيانات: {X.shape}")

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )

        return X_train, X_test, y_train, y_test

    def get_top_7_models(self):
        """ أفضل 7 نماذج متنوعة"""
        models = {
            # 1. Random Forest (أفضل نموذج تجميعي)
            'Random Forest': RandomForestClassifier(
                n_estimators=100,
                random_state=42,
                n_jobs=-1,
                max_depth=10
            ),

            # 2. XGBoost (أفضل خوارزمية boosting)
            'XGBoost': xgb.XGBClassifier(
                n_estimators=100,
                random_state=42,
                n_jobs=-1,
                max_depth=6,
                learning_rate=0.1
            ) if XGBOOST_AVAILABLE else None,

            # 3. Logistic Regression (نموذج خطي كلاسيكي)
            'Logistic Regression': LogisticRegression(
                random_state=42,
                max_iter=1000,
                n_jobs=-1,
                C=1.0
            ),

                        # 4. Neural Network (تعلم عميق)
            'Neural Network': MLPClassifier(
                hidden_layer_sizes=(100, 50),
                random_state=42,
                max_iter=500,
                alpha=0.001
            ),

            # 5. Gradient Boosting (boosting قوي)
            'Gradient Boosting': GradientBoostingClassifier(
                n_estimators=100,
                random_state=42,
                max_depth=5,
                learning_rate=0.1
            ),

            # 6. AdaBoost (متعزيز)
            'AdaBoost': AdaBoostClassifier(
                n_estimators=100,
                random_state=42
            ),

            # 7. K-Neighbors (نموذج قائم على المسافات)
            'K-Neighbors': KNeighborsClassifier(
                n_neighbors=5,
                n_jobs=-1
            )
        }

        # إزالة النماذج غير المتاحة
        return {name: model for name, model in models.items() if model is not None}

    def train_top_models(self, X_train, X_test, y_train, y_test):
        """تدريب  7 نماذج"""
        print("\n" + "="*60)
        print(" تدريب 7 نماذج متنوعة")

        models = self.get_top_7_models()
        results = {}

        for name, model in models.items():
            print(f"\n تدريب {name}...")

            try:
                start_time = time.time()

                # تدريب النموذج
                model.fit(X_train, y_train)
                training_time = time.time() - start_time

                # التنبؤ
                y_pred = model.predict(X_test)

                # حساب المقاييس
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred, zero_division=0)
                f1 = f1_score(y_test, y_pred, zero_division=0)

                # حساب AUC-ROC
                try:
                    if hasattr(model, 'predict_proba'):
                        y_proba = model.predict_proba(X_test)[:, 1]
                        auc_roc = roc_auc_score(y_test, y_proba)
                    else:
                        auc_roc = 0
                except:
                    auc_roc = 0

                # حساب Specificity
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

                results[name] = {
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'specificity': specificity,
                    'f1_score': f1,
                    'auc_roc': auc_roc,
                    'training_time': training_time,
                    'model': model
                }

                print(f"📊 ACCURACY: {accuracy:.4f}")
                print(f"🎯 F1: {f1:.4f}")
                print(f"📈 AUC: {auc_roc:.4f}")
                print(f"⏱️ TIME/s: {training_time:.2f}")

                # تحديث أفضل نموذج
                if accuracy > self.best_accuracy:
                    self.best_accuracy = accuracy
                    self.best_model = model
                    print(f"🏆أصبح أفضل نموذج!")

            except Exception as e:
                print(f"❌ خطأ في {name}: {e}")
                continue

        self.results = results

        if hasattr(self, 'pareto_selector'):
            best_name, _ = self.pareto_selector.choose_pareto_optimal(self.results)
            if best_name:
                self.best_model = self.results[best_name]['model']
                self.best_accuracy = self.results[best_name]['accuracy']

        return results

    def evaluate_results(self):
        """تقييم النتائج وعرض المقارنة"""
        print("\n" + "="*100)
        print("📊 النتائج النهائية 7 نماذج")
        print("="*100)

        if not self.results:
            print("❌ لا توجد نتائج لتقييمها!")
            return pd.DataFrame()

        # إنشاء DataFrame للنتائج
        results_data = []
        for name, metrics in self.results.items():
            results_data.append({
                'النموذج': name,
                'الدقة': metrics['accuracy'],
                'الدقة (Precision)': metrics['precision'],
                'الاستدعاء (Recall)': metrics['recall'],
                'نتيجة F1': metrics['f1_score'],
                'AUC-ROC': metrics['auc_roc'],
                'النوعية (Specificity)': metrics['specificity'],
                'وقت التدريب (ث)': metrics['training_time']
            })

        results_df = pd.DataFrame(results_data)

        # عرض النتائج مرتبة حسب الدقة
        print("\n🏆 ترتيب النماذج من الأفضل إلى الأسوأ:")
        sorted_results = results_df.sort_values('الدقة', ascending=False)

        for i, (_, row) in enumerate(sorted_results.iterrows(), 1):
            print(f"\n{i}. {row['النموذج']}:")
            print(f"الدقة: {row['الدقة']:.4f} | F1: {row['نتيجة F1']:.4f} | AUC: {row['AUC-ROC']:.4f}")
            print(f"الوقت: {row['وقت التدريب (ث)']:.2f}ثانية")

        print(f"📈 أفضل دقة: {self.best_accuracy:.4f}")

        return sorted_results
        #-----------------------------
class ParetoModelSelector:
    """اختيار أفضل نموذج باستخدام Pareto Optimal Front (الأفضل علمياً)"""

    def __init__(self, min_auc=0.7, min_recall=0.8):
        """
        min_auc: أقل AUC مقبول (افتراضي 0.7)
        min_recall: أقل Recall مقبول (افتراضي 0.8)
        """
        self.unified_selector = UnifiedModelSelector({
            'min_auc': min_auc,
            'min_recall': min_recall,
            'min_accuracy': 0.7,
            'min_f1': 0.7,
            'min_precision': 0.6
        })

    def choose_pareto_optimal(self, models_results):
        return self.unified_selector.choose_pareto_optimal(models_results)

# ================ خوارزمية ReservedIPPhishingDetector - نسخة المعدلة ================

class ReservedIPPhishingDetector:
    """خوارزمية متخصصة للكشف عن التصيد في عناوين IP المحجوزة - نسخة """

    def __init__(self):
        # نطاقات IP المحجوزة حسب RFC
        self.reserved_ranges = [
            # RFC 1918 - Private Networks
            ipaddress.ip_network('10.0.0.0/8'),
            ipaddress.ip_network('172.16.0.0/12'),
            ipaddress.ip_network('192.168.0.0/16'),

            # Loopback
            ipaddress.ip_network('127.0.0.0/8'),

            # Link-local
            ipaddress.ip_network('169.254.0.0/16'),

            # TEST-NET (RFC 5737)
            ipaddress.ip_network('192.0.2.0/24'),   # TEST-NET-1
            ipaddress.ip_network('198.51.100.0/24'), # TEST-NET-2
            ipaddress.ip_network('203.0.113.0/24'),  # TEST-NET-3

            # Multicast
            ipaddress.ip_network('224.0.0.0/4'),

            # Reserved
            ipaddress.ip_network('240.0.0.0/4'),

            # Other special ranges
            ipaddress.ip_network('0.0.0.0/8'),      # Current network
            ipaddress.ip_network('100.64.0.0/10'),  # CGNAT
            ipaddress.ip_network('192.0.0.0/24'),   # IETF Protocol Assignments
        ]

        # نمط محسّن لاستخراج IPv4
        self.ipv4_pattern = re.compile(r'\b(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(?::\d+)?\b')

        self.selected_model = None
        self.feature_names = []
        self.label_encoder = LabelEncoder()

        self.rules = {
            # قواعد أساسية قوية
            'admin_page': 0.9,           # ⬆️ أي صفحة إدارة = خطر عالي جداً
            'no_https': 0.8,             # ⬆️ لا HTTPS = خطر كبير
            'non_standard_port': 0.6,    # ⬆️ منفذ غير قياسي
            'suspicious_extension': 0.3,  # امتدادات خطرة
            'encoded_chars': 0.3,        # أحرف مشفرة
            'long_path': 0.2,            # مسار طويل

            # قواعد خاصة بـ IPs محجوزة
            'reserved_ip_with_admin': 1.2,  # ⬆️ إدارة على IP محجوز = خطر قصوى
            'localhost_access': 0.7,        # الوصول لـ localhost
            'private_ip_external': 0.8,     # IP خاص يظهر خارجي

            # تخفيضات طفيفة فقط
            'common_gateway': -0.3,        # ⬆️ تخفيض طفيف فقط (ليس -0.3)
            'standard_port': -0.1,        # منفذ قياسي يقلل الخطورة قليلاً

            # العتبة الذكية
            'phishing_threshold': 0.85,    # ⬆️ عتبة أعلى لكن النقاط أكبر
            'high_confidence_threshold': 1.2
        }

        #  قوائم دقيقة لصفحات الإدارة الخطرة
        self.high_risk_admin_pages = {
            'login': 0.9, 'admin': 0.9, 'dashboard': 0.8,
            'control': 0.8, 'config': 0.8, 'setup': 0.8,
            'administrator': 0.9, 'panel': 0.7, 'cp': 0.7,
            'root': 0.9, 'sysadmin': 0.9, 'manager': 0.7,
            'verify': 0.8, 'authenticate': 0.8, 'signin': 0.8,
            'password': 0.9, 'credential': 0.9, 'account': 0.7,
            'bank': 0.9, 'payment': 0.9, 'transfer': 0.8
        }
                #  **إضافة قواعد تلقائية صارمة**
        self.automatic_rules = {
            # القواعد التلقائية - كسرها = تصيد مؤكد
            'http_on_sensitive': 1.5,  # HTTP على صفحة حساسة
            'private_ip_public': 1.3,  # IP خاص يظهر للعامة
            'loopback_external': 1.4,  # Loopback خارجي
            'test_net_production': 1.2,  # TEST-NET في بيئة إنتاج
        }

        #  **تعديل حساب الثقة**
        self.confidence_calibration = {
            'min_confidence': 0.7,      # أقل ثقة مقبولة
            'good_confidence': 0.85,    # ثقة جيدة
            'high_confidence': 0.95,    # ثقة عالية
            'certain_confidence': 0.99, # ثقة مؤكدة
        }

        self.suspicious_extensions = ['.php', '.asp', '.aspx', '.jsp', '.cgi', '.pl']
        self.encoded_patterns = ['%', '@', '!', '*', '(', ')']

    def train_and_select_pareto(self, X_train, X_test, y_train, y_test):
        """تدريب واختيار باستخدام Pareto Optimal"""

        # 1. تدريب النماذج (استدعاء الدالة الأصلية)
        self.train_top_models(X_train, X_test, y_train, y_test)

        # 2. اختيار أفضل نموذج باستخدام Pareto
        best_name, details = self.pareto_selector.choose_pareto_optimal(self.results)

        if best_name:
            self.best_model = self.results[best_name]['model']
            self.best_accuracy = self.results[best_name]['accuracy']

        return best_name, details

    def is_reserved_ip(self, ip_str):
        """التحقق إذا كان العنوان IP محجوز"""
        try:
            ip = ipaddress.ip_address(ip_str.split(':')[0])  # إزالة المنفذ إن وجد
            for network in self.reserved_ranges:
                if ip in network:
                    return True
            return False
        except:
            return False

    def extract_ip_from_url(self, url):
        """استخراج عنوان IP من URL - الإصدار المحسّن"""
        # البحث عن IPv4 في النص
        match = self.ipv4_pattern.search(url)
        if match:
            return match.group()

        return None

    def extract_reserved_ip_features(self, url):
        """استخراج ميزات متقدمة للـ IPs المحجوزة فقط"""
        features = {}

        # استخراج IP من الرابط
        ip = self.extract_ip_from_url(url)

        if ip:
            features['has_ip'] = 1

            # تنظيف الـ IP من المنفذ
            ip_clean = ip.split(':')[0] if ':' in ip else ip

            try:
                ip_obj = ipaddress.ip_address(ip_clean)

                # 1. ميزات النوع
                features['is_rfc1918'] = 1 if self._is_rfc1918(ip_obj) else 0
                features['is_loopback'] = 1 if ip_obj.is_loopback else 0
                features['is_link_local'] = 1 if ip_obj.is_link_local else 0
                features['is_multicast'] = 1 if ip_obj.is_multicast else 0
                features['is_private'] = 1 if ip_obj.is_private else 0
                features['is_reserved'] = 1 if ip_obj.is_reserved else 0

                # 2. ميزات بناء على النطاق
                if isinstance(ip_obj, ipaddress.IPv4Address):
                    # تحويل إلى عدد صحيح للتحليل
                    ip_int = int(ip_obj)

                    # أخذ البايتات
                    first_byte = (ip_int >> 24) & 0xFF
                    second_byte = (ip_int >> 16) & 0xFF

                    features['ip_first_byte'] = first_byte
                    features['ip_second_byte'] = second_byte
                    features['ip_third_byte'] = (ip_int >> 8) & 0xFF
                    features['ip_fourth_byte'] = ip_int & 0xFF

                    # فئة الشبكة التقليدية
                    if first_byte <= 127:
                        features['ip_class'] = 1  # Class A
                    elif first_byte <= 191:
                        features['ip_class'] = 2  # Class B
                    elif first_byte <= 223:
                        features['ip_class'] = 3  # Class C
                    else:
                        features['ip_class'] = 4  # Class D/E

                    # هل هو عنوان شبكة أو بث؟
                    features['is_network_address'] = 1 if (ip_int & 0xFF) == 0 else 0
                    features['is_broadcast_address'] = 1 if (ip_int & 0xFF) == 255 else 0
            except:
                # في حالة خطأ، وضع القيم الافتراضية
                features['is_rfc1918'] = 0
                features['is_loopback'] = 0
                features['is_link_local'] = 0
                features['is_multicast'] = 0
                features['is_private'] = 0
                features['is_reserved'] = 0
                features['ip_first_byte'] = 0
                features['ip_second_byte'] = 0
                features['ip_third_byte'] = 0
                features['ip_fourth_byte'] = 0
                features['ip_class'] = 0
                features['is_network_address'] = 0
                features['is_broadcast_address'] = 0
        else:
            features['has_ip'] = 0
            # تعيين القيم الافتراضية
            features['is_rfc1918'] = 0
            features['is_loopback'] = 0
            features['is_link_local'] = 0
            features['is_multicast'] = 0
            features['is_private'] = 0
            features['is_reserved'] = 0
            features['ip_first_byte'] = 0
            features['ip_second_byte'] = 0
            features['ip_third_byte'] = 0
            features['ip_fourth_byte'] = 0
            features['ip_class'] = 0
            features['is_network_address'] = 0
            features['is_broadcast_address'] = 0

        # 3. ميزات الرابط نفسه
        url_lower = url.lower()

        # أطوال الأجزاء
        parsed_url = urlparse(url) if '://' in url else urlparse('http://' + url)
        hostname = parsed_url.netloc

        features['url_length'] = len(url)
        features['hostname_length'] = len(hostname)
        features['path_length'] = len(parsed_url.path)

        # وجود منفذ
        features['has_port'] = 1 if ':' in hostname and hostname.count(':') == 1 else 0

        # وجود مسار مشبوه
        suspicious_paths = ['login', 'admin', 'secure', 'verify', 'account', 'bank']
        features['suspicious_path'] = 0
        for path in suspicious_paths:
            if path in parsed_url.path.lower():
                features['suspicious_path'] = 1
                break

        # استخدام HTTPS
        features['has_https'] = 1 if parsed_url.scheme == 'https' else 0

        # وجود معلمات
        features['has_query'] = 1 if parsed_url.query else 0

        # نسبة الأرقام في الـ hostname
        if hostname:
            digits = sum(c.isdigit() for c in hostname)
            features['digit_ratio'] = digits / len(hostname) if len(hostname) > 0 else 0
        else:
            features['digit_ratio'] = 0

        # 4. ميزات السياق (لـ IPs المحجوزة)
        features['is_common_gateway'] = self._is_common_gateway(ip_clean) if ip else 0
        features['is_common_admin'] = self._is_common_admin_page(parsed_url.path) if ip else 0

        return features

    def _is_rfc1918(self, ip_obj):
        """التحقق إذا كان IP ضمن RFC 1918"""
        if isinstance(ip_obj, ipaddress.IPv4Address):
            ip_int = int(ip_obj)
            # 10.0.0.0/8
            if (ip_int >> 24) == 10:
                return True
            # 172.16.0.0/12
            if (ip_int >> 20) == 0xAC1:
                return True
            # 192.168.0.0/16
            if (ip_int >> 16) == 0xC0A8:
                return True
        return False

    def _is_common_gateway(self, ip):
        """التحقق إذا كان IP بوابة شائعة"""
        common_gateways = [
            '192.168.0.1', '192.168.1.1', '192.168.1.254',
            '10.0.0.1', '10.0.0.138',
            '172.16.0.1', '172.16.1.1',
            '192.168.100.1', '192.168.123.254'
        ]
        return 1 if ip in common_gateways else 0

    def _is_common_admin_page(self, path):
        """التحقق إذا كان المسار صفحة إدارة شائعة"""
        admin_pages = [
            '/admin', '/login', '/config', '/setup',
            '/router', '/gateway', '/administrator',
            '/cp', '/controlpanel', '/webadmin'
        ]
        path_lower = path.lower()
        return 1 if any(page in path_lower for page in admin_pages) else 0

    def filter_reserved_ips(self, df):
        """تصفية البيانات لاستخراج الروابط مع IPs محجوزة فقط"""
        reserved_urls = []
        reserved_labels = []

        for idx, row in df.iterrows():
            url = row['url']
            ip = self.extract_ip_from_url(url)

            if ip and self.is_reserved_ip(ip):
                reserved_urls.append(url)
                if 'result' in row:
                    reserved_labels.append(row['result'])
                elif 'label' in row:
                    reserved_labels.append(row['label'])
                else:
                    reserved_labels.append(-1)  # علامة غير معروفة

        return pd.DataFrame({
            'url': reserved_urls,
            'label': reserved_labels
        })


    def predict_reserved_ip(self, url):
        """التنبؤ إذا كان رابط الـ IP المحجوز ضاراً"""
        # التحقق أولاً إذا كان يحتوي على IP محجوز
        ip = self.extract_ip_from_url(url)

        if not ip or not self.is_reserved_ip(ip):
            return {
                'is_reserved_ip': False,
                'prediction': 'normal',
                'confidence': 0.0,
                'reason': 'لا يحتوي على IP محجوز'
            }

        # استخراج الميزات
        features = self.extract_reserved_ip_features(url)
        features_df = pd.DataFrame([features])

        # التأكد من وجود جميع الميزات المطلوبة
        if not self.feature_names:
            self.feature_names = features_df.columns.tolist()

        # ملء القيم الناقصة
        for col in self.feature_names:
            if col not in features_df.columns:
                features_df[col] = 0

        # إعادة ترتيب الأعمدة
        features_df = features_df[self.feature_names]

        # التنبؤ باستخدام النموذج المدرب
        if self.selected_model:
            try:
                # التنبؤ واحتمالية التنبؤ
                prediction = self.selected_model.predict(features_df)[0]

                if hasattr(self.selected_model, 'predict_proba'):
                    proba = self.selected_model.predict_proba(features_df)[0]
                    confidence = proba[1] if prediction == 1 else proba[0]
                else:
                    confidence = 0.5

                result = 'phishing' if prediction == 1 else 'benign'

            except Exception as e:
                print(f"⚠️  خطأ في التنبؤ: {e}")
                return self.rule_based_detection(url, ip)
        else:
            # إذا لم يتم تدريب النموذج، استخدم القواعد البسيطة
            return self.rule_based_detection(url, ip)

        # تفسير النتيجة
        reasons = self.explain_prediction(features, url, ip)

        return {
            'is_reserved_ip': True,
            'ip_address': ip,
            'prediction': result,
            'confidence': confidence,
            'reasons': reasons
        }

    def explain_prediction(self, features, url, ip):
        """تفسير سبب القرار"""
        reasons = []

        if features.get('is_rfc1918', 0) == 1:
            reasons.append("IP خاص (RFC 1918)")

        if features.get('suspicious_path', 0) == 1:
            reasons.append("مسار مشبوه (login/admin)")

        if features.get('has_https', 0) == 0:
            reasons.append("غير آمن (HTTP فقط)")

        if features.get('is_common_gateway', 0) == 1:
            reasons.append("IP بوابة شائعة")

        if features.get('is_common_admin', 0) == 1:
            reasons.append("صفحة إدارة شائعة")

        if len(reasons) == 0:
            reasons.append("خصائص IP محجوز عامة")

        return reasons

    def detect_phishing_by_rules_enhanced(self, url):
        # استخراج وتحليل الرابط
        print(f"IPs: {url[:60]}...")
        parsed = urlparse(url) if '://' in url else urlparse('http://' + url)
        hostname = parsed.netloc
        path = parsed.path.lower()
        query = parsed.query.lower()
        full_path = path + ('?' + query if query else '')

        ip = self.extract_ip_from_url(url)

        if not ip or not self.is_reserved_ip(ip):
            return {'prediction': 'normal', 'confidence': 0.0}

        reasons = []
        score = 0
        warning_flags = []

        #  1. تحليل صفحة الإدارة (الأهم)
        admin_detected = False
        admin_score = 0
        max_admin_score = 0

        for admin_page, weight in self.high_risk_admin_pages.items():
            if admin_page in full_path:
                admin_detected = True
                admin_score = max(admin_score, weight)
                max_admin_score = max(max_admin_score, weight)
                reasons.append(f"صفحة {admin_page} على IP محجوز ({weight})")
                warning_flags.append(f"⚠️ {admin_page.upper()}")

        if admin_detected:
            score += self.rules['reserved_ip_with_admin'] + max_admin_score
            reasons.append(f"إدارة على IP محجوز ({self.rules['reserved_ip_with_admin']} + {max_admin_score})")

        #  2. تحليل البروتوكول
        if not url.startswith('https'):
            score += self.rules['no_https']
            reasons.append(f"لا HTTPS ({self.rules['no_https']})")
            warning_flags.append("⚠️ HTTP فقط")

        #  3. تحليل المنفذ
        standard_ports = [80, 443]
        port_match = re.search(r':(\d+)(?=/|$)', hostname)
        if port_match:
            port = int(port_match.group(1))
            if port not in standard_ports:
                score += self.rules['non_standard_port']
                reasons.append(f"منفذ غير قياسي ({port}: {self.rules['non_standard_port']})")
                warning_flags.append(f"⚠️ منفذ {port}")
            else:
                score += self.rules['standard_port']
                reasons.append(f"منفذ قياسي ({self.rules['standard_port']})")

        #  4. تحليل الامتدادات الخطرة
        for ext in self.suspicious_extensions:
            if ext in path:
                score += self.rules['suspicious_extension']
                reasons.append(f"امتداد خطر {ext} ({self.rules['suspicious_extension']})")
                warning_flags.append(f"⚠️ {ext}")
                break

        #  5. تحليل الأحرف المشفرة
        for pattern in self.encoded_patterns:
            if pattern in url:
                score += self.rules['encoded_chars']
                reasons.append(f"أحرف مشفرة ({self.rules['encoded_chars']})")
                warning_flags.append("⚠️ تشفير")
                break

        #  6. تحليل طول المسار
        if len(path) > 30:
            score += self.rules['long_path']
            reasons.append(f"مسار طويل ({self.rules['long_path']})")

        #  7. تحليل بوابة شائعة (تخفيض بسيط فقط)
        common_gateways = ['192.168.1.1', '192.168.0.1', '10.0.0.1']
        ip_without_port = ip.split(':')[0]
        if ip_without_port in common_gateways:
            score += self.rules['common_gateway']
            reasons.append(f"بوابة شائعة ({self.rules['common_gateway']})")

        #  8. IPs عالية الخطورة
        high_risk_ips = ['169.254.', '224.', '240.', '0.0.0.0', '127.0.0.1', 'localhost']
        if any(risk_ip in url for risk_ip in high_risk_ips):
            additional_score = 0.5 if 'localhost' in url else 0.3
            score += additional_score
            reasons.append(f"IP عالي الخطورة (+{additional_score})")
            warning_flags.append("🚨 IP عالي الخطورة")

        #  9. عوامل إضافية
        # وجود معلمات
        if '?' in url:
            param_score = 0.2
            score += param_score
            reasons.append(f"وجود معلمات (+{param_score})")

        # وجود رموز خاصة
        special_chars = ['@', '!', '#', '$', '%', '&', '*']
        if any(char in url for char in special_chars):
            score += 0.2
            reasons.append("رموز خاصة (+0.2)")

        #  تحديد العتبة الذكية
        if admin_detected:
            threshold = 0.8  # إذا فيه إدارة، نرفع العتبة
        else:
            threshold = self.rules['phishing_threshold']

        #  حساب الثقة الذكية
        if admin_detected and max_admin_score >= 0.8:
            # صفحات إدارة خطرة جداً
            base_confidence = 0.85
        elif admin_detected:
            # صفحات إدارة عادية
            base_confidence = 0.75
        elif score > 1.5:
            # نقاط عالية جداً
            base_confidence = 0.9
        elif score > 1.0:
            base_confidence = 0.8
        elif score > 0.6:
            base_confidence = 0.7
        else:
            base_confidence = 0.5

        # زيادة الثقة مع زيادة النقاط
        confidence = min(base_confidence + (score * 0.1), 0.98)

        # 🔥 تحديد النتيجة النهائية
        if score >= threshold:
            result = 'PHISHING'
            if score >= 2.0:
                severity = "🚨(HIGH RISK)"
            elif score >= 1.5:
                severity = "🔴(RISK)"
            else:
                severity = "🟠(SUSPICIOUS)"
        else:
            result = 'BENIGN'
            if score <= 0.3:
                severity = "🟢(SAFE)"
            elif score <= 0.6:
                severity = "🟡(NEUTRAL)"
            else:
                severity = "🟠(LOW RISK)"

        # 🔥 عرض النتائج
        print(f"📊النتيجة: {result}")
        print(f"⚠️الشدة: {severity}")
        print(f"🎯النقاط: {score:.2f} (العتبة:{threshold:.2f})")
        print(f"🎯الثقة: {confidence:.2%}")
        print(f"📍IP: {ip}")
        if warning_flags:
            print(f"🚩التحذيرات: {', '.join(warning_flags[:3])}")
        if reasons:
            print(f"📝أهم الأسباب:")
            for i, reason in enumerate(reasons[:4], 1):
                print(f"{i}.{reason}")

        print("\n")

        return {
            'is_reserved_ip': True,
            'ip_address': ip,
            'prediction': result,
            'confidence': confidence,
            'severity': severity,
            'score': score,
            'reasons': reasons[:6],  # أول 6 أسباب فقط
            'warning_flags': warning_flags,
            'rule_based': True,
            'version': ' - عالي الدقة'
        }

    def detect_phishing_by_rules_enhanced_v2(self, url):
        return self.detect_phishing_by_rules_enhanced(url)


    def _analyze_smart_context(self, url):
        """تحليل ذكي للسياق لتقليل False Positives"""
        parsed = urlparse(url) if '://' in url else urlparse('http://' + url)
        path = parsed.path.lower()
        hostname = parsed.netloc

        score = 0.5  # نقطة بداية محايدة

        # عوامل تقليل الخطورة (سلبية)
        # 1. IPs بوابة شائعة
        common_gateways = ['192.168.1.1', '192.168.0.1', '10.0.0.1', '192.168.100.1']
        ip = self.extract_ip_from_url(url)
        if ip:
            ip_base = ip.split(':')[0]
            if ip_base in common_gateways:
                score -= 0.4  # تقليل كبير

        # 2. مسارات نظام آمنة
        safe_paths = ['/status', '/info', '/help', '/diagnostics', '/wlan', '/wireless',
                     '/network', '/system', '/about', '/contact']
        for safe_path in safe_paths:
            if safe_path in path:
                score -= 0.3
                break

        # 3. امتدادات آمنة
        safe_extensions = ['.htm', '.html', '.cgi']
        for ext in safe_extensions:
            if path.endswith(ext):
                score -= 0.2
                break

        # 4. منفذ قياسي
        if ':80' in hostname or ':443' in hostname:
            score -= 0.1

        # عوامل زيادة الخطورة (إيجابية)
        # 1. صفحات حساسة
        sensitive_pages = ['/login', '/admin', '/config', '/setup', '/password',
                          '/verify', '/authenticate', '/bank', '/payment']
        for sensitive in sensitive_pages:
            if sensitive in path:
                score += 0.3
                break

        # 2. معلمات خطرة
        query = parsed.query.lower()
        risky_params = ['token=', 'auth=', 'key=', 'secret=', 'password=', 'redirect=']
        for param in risky_params:
            if param in query:
                score += 0.2
                break

        return max(0, min(1, score))
        # ================ دوال التشغيل المحسنة ================
def run_reserved_detector_unified(show_first_n=10, output_file=None):
    detector = ReservedIPPhishingDetector()

    try:
        # تحميل البيانات
        new_data = pd.read_csv(RESERVED_ONLY_DATA_FILE)

        # العثور على عمود URL
        url_col = None
        for col in new_data.columns:
            if 'url' in col.lower():
                url_col = col
                break

        if not url_col:
            print("❌ لم يتم العثور على عمود URL")
            return None

        urls = new_data[url_col].astype(str).tolist()
        print(f"✅ تم تحميل {len(urls)} رابط")

        # قائمة لتخزين جميع النتائج
        all_results = []

        # 2️⃣ تحليل باقي الروابط بصمت
        if len(urls) > show_first_n:
            remaining_count = len(urls) - show_first_n

            for i in range(show_first_n, len(urls)):
                url = urls[i]

                # 🔇 تحليل بدون طباعة أي شيء
                import sys, io
                old_stdout = sys.stdout
                sys.stdout = io.StringIO()

                try:
                    result = detector.detect_phishing_by_rules_enhanced_v2(url)
                finally:
                    sys.stdout = old_stdout


                # حفظ النتيجة
                result_entry = {
                    'index': i + 1,
                    'url': url,
                    'prediction': result['prediction'],
                    'confidence': result['confidence'],
                    'score': result.get('score', 0),
                    'severity': result.get('severity', 'UNKNOWN'),
                    'warning_flags': ' | '.join(result.get('warning_flags', []))[:100],
                    'top_reasons': ' | '.join(result.get('reasons', []))[:100],
                }

                all_results.append(result_entry)

        # 3️⃣ تحويل النتائج إلى DataFrame
        results_df = pd.DataFrame(all_results)

        total = len(results_df)
        if total > 0:
            phishing_count = sum(results_df['prediction'] == 'PHISHING')
            benign_count = sum(results_df['prediction'] == 'BENIGN')
            error_count = sum(results_df['prediction'] == 'ERROR')

            print(f"\n📊 إجمالي الروابط المحللة: {total}")
            print(f"   • PHISHING: {phishing_count} ({phishing_count/total*100:.1f}%)")
            print(f"   • BENIGN: {benign_count} ({benign_count/total*100:.1f}%)")
            if error_count > 0:
                print(f"   • ERROR: {error_count} ({error_count/total*100:.1f}%)")

            # توزيع الشدة
            print(f"\n⚠️توزيع الشدة:")
            severity_counts = results_df['severity'].value_counts()
            for severity, count in severity_counts.items():
                print(f"   • {severity}: {count} رابط ({count/total*100:.1f}%)")

            # متوسط الثقة
            if 'confidence' in results_df.columns:
                avg_confidence = results_df['confidence'].mean()
                print(f"\n🎯 متوسط الثقة: {avg_confidence:.1%}")

        # 5️⃣ حفظ النتائج في ملف
        from datetime import datetime

        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"/content/reserved_results_unified_{timestamp}.csv"

        results_df.to_csv(output_file, index=False, encoding='utf-8-sig')

        print(f"\n💾 تم حفظ جميع النتائج في: {output_file}")

        # 7️⃣ إنشاء ملخص إحصائي إضافي
        summary_file = output_file.replace('.csv', '_summary.txt')
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write("="*60 + "\n")
            f.write("ملخص نتائج خوارزمية IPs المحجوزة\n")
            f.write("="*60 + "\n\n")
            f.write(f"تاريخ التحليل: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"إجمالي الروابط: {total}\n")
            f.write(f"عدد PHISHING: {phishing_count} ({phishing_count/total*100:.1f}%)\n")
            f.write(f"عدد BENIGN: {benign_count} ({benign_count/total*100:.1f}%)\n")
            f.write(f"عدد الأخطاء: {error_count}\n")
            f.write(f"متوسط الثقة: {avg_confidence:.1%}\n\n")

            f.write("توزيع الشدة:\n")
            for severity, count in severity_counts.items():
                f.write(f"  • {severity}: {count} ({count/total*100:.1f}%)\n")

        print(f"\n📄 تم إنشاء ملخص إحصائي في: {summary_file}")

        return results_df

    except FileNotFoundError:
        print(f"❌ ملف الداتا غير موجود: {RESERVED_ONLY_DATA_FILE}")
        return None
    except Exception as e:
        print(f"❌ حدث خطأ: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
# دالة اختيارية للتوافق مع الكود
def run_reserved_detector_silent():
    """تشغيل الخوارزمية بدون طباعة تفاصيل كل الروابط (للتوافق)"""
    return run_reserved_detector_unified()
# دالة اختيارية أخرى للتوافق
def run_reserved_detector_on_new_data_complete():
    """تشغيل الخوارزمية مع تخزين جميع النتائج (للتوابق)"""
    return run_reserved_detector_unified()
# ================ الفئة المدمجة المعدلة ================

class CompletePhishingDetectionSystem:
    """نظام كامل للكشف عن التصيد مع فصل الـ IPs المحجوزة"""

    def __init__(self):
        self.top7_classifier = Top7ModelsClassifier()
        self.reserved_ip_detector = ReservedIPPhishingDetector()
        self.is_reserved_model_trained = False

    def extract_only_real_reserved_ips(self, data):
        """استخراج IPs محجوزة حقيقية فقط - الطريقة المحسنة"""

        try:
           # نطاقات IPs المحجوزة الحقيقية
           reserved_patterns = [
               r'^https?://192\.168\.\d{1,3}\.\d{1,3}',
               r'^https?://10\.\d{1,3}\.\d{1,3}\.\d{1,3}',
               r'^https?://172\.(1[6-9]|2[0-9]|3[0-1])\.\d{1,3}\.\d{1,3}',
               r'^https?://127\.0\.0\.\d{1,3}',
               r'^https?://169\.254\.\d{1,3}\.\d{1,3}',
               r'^https?://192\.0\.2\.\d{1,3}',
               r'^https?://198\.51\.100\.\d{1,3}',
               r'^https?://203\.0\.113\.\d{1,3}',
               r'^https?://224\.\d{1,3}\.\d{1,3}\.\d{1,3}',
               r'^https?://240\.\d{1,3}\.\d{1,3}\.\d{1,3}',
          ]

           combined_pattern = '|'.join(reserved_patterns)

           # البحث في عمود URL
           url_col = None
           for col in data.columns:
               if 'url' in col.lower():
                   url_col = col
                   break

           if not url_col:
               print("❌ لم يتم العثور على عمود URL")
               return pd.DataFrame(), pd.DataFrame()

           # التأكد من أن البيانات ليست فارغة
           if len(data) == 0:
              print("❌ البيانات فارغة!")
              return pd.DataFrame(), pd.DataFrame()

           # إنشاء mask بشكل آمن
           try:
               # تحويل القيم إلى سلسلة نصية والتعامل مع القيم الفارغة
               url_series = data[url_col].astype(str)

               # تطبيق البحث بالنمط
               mask = url_series.str.contains(combined_pattern, regex=True, na=False)

               # التحقق من أن الـ mask ليس فارغاً
               if mask.empty:
                   print("⚠️ الـ mask فارغ - لا توجد روابط مطابقة")
                   return pd.DataFrame(), data.copy()

               # استخدام الـ mask لتصفية البيانات
               real_reserved = data[mask].copy()
               normal_urls = data[~mask].copy()

               return real_reserved, normal_urls

           except Exception as mask_error:
               print(f"❌ خطأ في إنشاء الـ mask: {mask_error}")
               print(f"   نوع البيانات: {type(data)}")
               return pd.DataFrame(), data.copy()

        except Exception as e:
           print(f"❌ خطأ غير متوقع في extract_only_real_reserved_ips: {e}")
           import traceback
           traceback.print_exc()
           return pd.DataFrame(), pd.DataFrame()

    def load_and_split_data(self, data_file):
        """تحميل وتقسيم البيانات حسب نوع الرابط"""
        print(f"📂 جاري تحميل البيانات من: {data_file}")

        try:
           # تحميل البيانات مع معالجة الأخطاء
           try:
               data = pd.read_csv(data_file)
           except Exception as read_error:
               print(f"❌ خطأ في قراءة الملف: {read_error}")
               return {
                   'all_data': pd.DataFrame(),
                   'normal_data': pd.DataFrame(),
                   'reserved_data': pd.DataFrame()
               }

           if len(data) == 0:
               print("❌ الملف فارغ!")
               return {
                   'all_data': pd.DataFrame(),
                   'normal_data': pd.DataFrame(),
                   'reserved_data': pd.DataFrame()
               }

           print(f"✅ تم تحميل {len(data)} سجل")
           print(f"📊 أعمدة البيانات: {list(data.columns)}")

           # استخدام الطريقة المحسنة للتقسيم
           reserved_data, normal_data = self.extract_only_real_reserved_ips(data)

           print(f"✅ الروابط العادية: {len(normal_data)}")
           print(f"✅ روابط IPs المحجوزة: {len(reserved_data)}")

           return {
               'all_data': data,
               'normal_data': normal_data,
               'reserved_data': reserved_data
           }

        except Exception as e:
           print(f"❌ خطأ في تحميل البيانات: {e}")
           import traceback
           traceback.print_exc()
           return {
               'all_data': pd.DataFrame(),
               'normal_data': pd.DataFrame(),
               'reserved_data': pd.DataFrame()
        }
        self.data_split_cache = result
        return result

    def train_system(self, data_file):
        """تدريب النظام الكامل"""
        if not hasattr(self, 'data_split') or self.data_split is None:
           data_split = self.load_and_split_data(data_file)

        # 2. تدريب النماذج السبعة على الروابط العادية فقط
        if len(data_split['normal_data']) > 0:
            self.top7_classifier.raw_data = data_split['normal_data']
            self.top7_classifier.processed_data, self.top7_classifier.feature_names = \
                self.top7_classifier.data_processor.prepare_data_fast(data_split['normal_data'])

            if len(self.top7_classifier.processed_data) > 0:
              #تقسيم البيانات
                X_train, X_test, y_train, y_test = self.top7_classifier.split_data()
                self.top7_classifier.train_top_models(X_train, X_test, y_train, y_test)
                self.top7_classifier.evaluate_results()

            else:
                print("⚠️  لا توجد بيانات كافية للتدريب")
        else:
            print("⚠️  لا توجد روابط عادية للتدريب")

        if len(data_split['reserved_data']) > 0:

            self.reserved_data = data_split['reserved_data']
            self.is_reserved_model_trained = True
        else:
            print("⚠️  لا توجد روابط IPs محجوزة")
            self.is_reserved_model_trained = False

        print("=" * 60)

        return {
            'top7_results': self.top7_classifier.results,
            'reserved_ip_trained': self.is_reserved_model_trained,
            'data_stats': {
                'total': len(data_split['all_data']),
                'normal': len(data_split['normal_data']),
                'reserved': len(data_split['reserved_data'])
            }
        }

    def predict_url(self, url):
        """التنبؤ برابط جديد"""
        print(f"\n🔍 تحليل الرابط: {url}")

        # استخدام الطريقة المحسنة للتحقق من IPs محجوزة
        reserved_patterns = [
            r'^https?://192\.168\.\d{1,3}\.\d{1,3}',
            r'^https?://10\.\d{1,3}\.\d{1,3}\.\d{1,3}',
            r'^https?://172\.(1[6-9]|2[0-9]|3[0-1])\.\d{1,3}\.\d{1,3}',
            r'^https?://127\.0\.0\.\d{1,3}',
            r'^https?://169\.254\.\d{1,3}\.\d{1,3}',
        ]

        combined_pattern = '|'.join(reserved_patterns)
        is_reserved = bool(re.match(combined_pattern, url))

        if is_reserved:
            print("📍 الرابط يحتوي على IP محجوز حقيقي")
            print("  استخدام خوارزمية القواعد")

            # استخدام الخوارزمية المحسنة
            result = self.reserved_ip_detector.detect_phishing_by_rules_enhanced(url)

            return {
                'type': 'reserved_ip',
                'prediction': result['prediction'],
                'confidence': result['confidence'],
                'severity': result['severity'],
                'score': result['score'],
                'used_model': 'خوارزميةالقواعد'
            }
        else:
            print("🌐 الرابط عادي (لا يحتوي على IP محجوز)")
            if not hasattr(self.top7_classifier, 'best_model') or self.top7_classifier.best_model is None:
               print("⚠️ النموذج غير مدرب، استخدام خوارزمية القواعد كبديل")
               # استخدام الخوارزمية كبديل
               result = self.reserved_ip_detector.detect_phishing_by_rules_enhanced(url)
               return {
                  'type': 'normal_url_fallback',
                  'prediction': result['prediction'],
                  'confidence': result['confidence'],
                  'severity': result['severity'],
                  'score': result['score'],
                  'used_model': 'خوارزمية القواعد (بديل)'
                 }
            print(f"استخدام أفضل نموذج من السبعة")

            # استخدام النماذج السبعة
            if hasattr(self.top7_classifier, 'best_model') and self.top7_classifier.best_model is not None:
                try:
                    # استخراج الميزات
                    feature_extractor = OptimizedURLFeatureExtractor()
                    features = feature_extractor.extract_features_optimized(url)
                    features_df = pd.DataFrame([features])

                    # معالجة عمود tld كما في التدريب
                    if 'tld' in features_df.columns:
                        features_df['tld_encoded'] = features_df['tld'].astype('category').cat.codes
                        features_df = features_df.drop('tld', axis=1)

                    # التأكد من نفس أعمدة التدريب
                    if hasattr(self.top7_classifier, 'feature_names'):
                        for col in self.top7_classifier.feature_names:
                            if col not in features_df.columns:
                                features_df[col] = 0
                        features_df = features_df[self.top7_classifier.feature_names]

                    # استخدام أفضل نموذج
                    prediction = self.top7_classifier.best_model.predict(features_df)[0]

                    if hasattr(self.top7_classifier.best_model, 'predict_proba'):
                        proba = self.top7_classifier.best_model.predict_proba(features_df)[0]
                        confidence = proba[1] if prediction == 1 else proba[0]
                    else:
                        confidence = 0.5

                    result = 'phishing' if prediction == 1 else 'benign'

                    print(f"📊 النتيجة: {result.upper()}")
                    print(f"🎯 الثقة: {confidence:.2%}")
                    print(f"🤖 النموذج المستخدم: {self.top7_classifier.best_model.__class__.__name__}")

                    return {
                        'type': 'normal_url',
                        'prediction': result,
                        'confidence': confidence,
                        'used_model': self.top7_classifier.best_model.__class__.__name__
                    }
                except Exception as e:
                    print(f"⚠️خطأ في استخدام النموذج: {e}")
                    default_result = 'benign' if 'https' in url else 'suspicious'
                    return {
                        'type': 'normal_url',
                        'prediction': default_result,
                        'confidence': 0.5,
                        'used_model': 'Default (fallback)'
                    }
            else:
                default_result = 'benign' if 'https' in url else 'suspicious'
                print(f"📊 النتيجة (افتراضية): {default_result.upper()}")
                return {
                    'type': 'normal_url',
                    'prediction': default_result,
                    'confidence': 0.5,
                    'used_model': 'Default'
                }

    def analyze_and_test_reserved_ips(self, data_split):
        """🔬 الدالة الجديدة: تحليل وتقييم أداء خوارزمية كشف التصيد على IPs المحجوزة"""
        print("IPs تطبيق وتقييم الخوارزمية على ")

        reserved_data = data_split['reserved_data']

        if len(reserved_data) == 0:
            print("⚠️  لا توجد بيانات IPs محجوزة لتحليلها.")
            return None

        # 1. تحضير البيانات: استخراج الروابط والتسميات الحقيقية
        urls = reserved_data['url'].tolist()

        # الحصول على التسمية الحقيقية
        true_label_col = None
        for col in ['result', 'label', 'type']:
            if col in reserved_data.columns:
                true_label_col = col
                break

        if true_label_col:
            true_labels = reserved_data[true_label_col].tolist()
            # تحويل التسميات إلى 0/1 إذا كانت نصية
            if isinstance(true_labels[0], str):
                true_labels = [1 if label in ['malicious', 'phishing', '1'] else 0 for label in true_labels]
            print(f"✅ تم العثور على {len([x for x in true_labels if x==1])} تسمية تصيد")
        else:
            true_labels = None
            print("ℹ️  لم يتم العثور على تسميات حقيقية")
        # 2. تطبيق خوارزمية الكشف على كل رابط محجوز
        all_results = []

        for i, url in enumerate(urls):
            if i % 20 == 0:  # رسالة تقدم كل 20 رابط
                print(f"🔍 جاري تحليل الرابط {i+1}/{len(urls)}...")

            # استخدام الخوارزمية القاعدية المحسنة للكشف
            result = self.reserved_ip_detector.detect_phishing_by_rules_enhanced(url)

            # تجميع النتيجة
            result_entry = {
                'url': url,
                'prediction': result['prediction'],
                'confidence': result['confidence'],
                'score': result['score'],
                'severity': result['severity']
            }

            # إذا كانت التسمية الحقيقية متوفرة
            if true_labels is not None and i < len(true_labels):
                true_label = true_labels[i]
                if true_label != -1:  # تجاهل القيم غير المعروفة (-1)
                    # تحويل التسمية الرقمية إلى نصي للمقارنة
                    true_label_str = 'PHISHING' if true_label == 1 else 'BENIGN'
                    result_entry['true_label'] = true_label_str
                    result_entry['correct'] = (result['prediction'] == true_label_str)

            all_results.append(result_entry)

        # 3. تحويل النتائج إلى DataFrame لعرضها
        results_df = pd.DataFrame(all_results)

        # ب. عرض إحصائيات عامة
        total_count = len(results_df)
        phishing_count = sum(results_df['prediction'] == 'PHISHING')
        benign_count = sum(results_df['prediction'] == 'BENIGN')

        return results_df
    #-------------------------------------------------------------------------

class ReservedIPsXGBoostModel:
    """نموذج XGBoost متخصص لـ IPs المحجوزة - يتم تدريبه من الصفر"""

    def __init__(self):
        self.xgb_model = None
        self.feature_extractor = OptimizedURLFeatureExtractor()
        self.label_encoder = LabelEncoder()
        self.scaler = StandardScaler()
        self.is_trained = False
        self.feature_names = []
        self.model_performance = {}

    def prepare_reserved_ip_features(self, url):
        """تحضير ميزات خاصة بـ IPs المحجوزة"""
        features = {}

        # 1. استخراج IP
        ip_pattern = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}(?::\d+)?\b')
        ip_match = ip_pattern.search(url)
        ip = ip_match.group() if ip_match else None

        # 2. ميزات أساسية
        parsed = urlparse(url) if '://' in url else urlparse('http://' + url)
        url_lower = url.lower()
        path_lower = parsed.path.lower()
        query_lower = parsed.query.lower()

        features['url_length'] = len(url)
        features['has_https'] = 1 if parsed.scheme == 'https' else 0
        features['path_length'] = len(parsed.path)
        features['num_params'] = len(parsed.query.split('&')) if parsed.query else 0
        features['has_port'] = 1 if ':' in parsed.netloc else 0
        features['has_at_symbol'] = 1 if '@' in url else 0
        features['num_dots'] = url.count('.')
        features['num_digits'] = sum(c.isdigit() for c in url)

        # 3. ميزات IP محددة
        if ip:
            ip_clean = ip.split(':')[0] if ':' in ip else ip
            features['has_ip'] = 1

            # تحليل أجزاء IP
            try:
                ip_obj = ipaddress.ip_address(ip_clean)
                features['is_private'] = 1 if ip_obj.is_private else 0
                features['is_loopback'] = 1 if ip_obj.is_loopback else 0
                features['is_reserved'] = 1 if ip_obj.is_reserved else 0
                features['is_multicast'] = 1 if ip_obj.is_multicast else 0

                # أجزاء IP
                if isinstance(ip_obj, ipaddress.IPv4Address):
                    parts = ip_clean.split('.')
                    for i, part in enumerate(parts[:4], 1):
                        features[f'ip_part_{i}'] = int(part) if part.isdigit() else 0
                else:
                    for i in range(1, 5):
                        features[f'ip_part_{i}'] = 0
            except:
                features['is_private'] = 0
                features['is_loopback'] = 0
                features['is_reserved'] = 0
                features['is_multicast'] = 0
                for i in range(1, 5):
                    features[f'ip_part_{i}'] = 0
        else:
            features['has_ip'] = 0
            features['is_private'] = 0
            features['is_loopback'] = 0
            features['is_reserved'] = 0
            features['is_multicast'] = 0
            for i in range(1, 5):
                features[f'ip_part_{i}'] = 0

        # 4. ميزات محتوى
        dangerous_keywords = ['login', 'admin', 'password', 'bank', 'secure', 'verify',
                             'account', 'payment', 'transfer', 'credential', 'authenticate']
        features['dangerous_kw_count'] = sum(1 for kw in dangerous_keywords if kw in url_lower)

        admin_pages = ['/admin', '/login', '/cp', '/control', '/dashboard', '/config', '/setup']
        features['has_admin_page'] = 1 if any(page in path_lower for page in admin_pages) else 0

        risky_params = ['password=', 'token=', 'key=', 'secret=', 'auth=', 'redirect=']
        features['has_risky_params'] = 1 if any(param in query_lower for param in risky_params) else 0

        risky_extensions = ['.php', '.asp', '.aspx', '.jsp', '.cgi', '.pl']
        features['has_risky_extension'] = 1 if any(ext in path_lower for ext in risky_extensions) else 0

        # 5. ميزات إضافية
        features['suspicious_chars'] = sum(1 for c in url if c in ['@', '!', '#', '$', '%', '&', '*'])
        features['has_redirect'] = 1 if 'redirect' in url_lower or 'url=' in query_lower else 0

        return features

    def train_on_reserved_data(self, data_path, test_size=0.3):
        """تدريب النموذج على بيانات IPs المحجوزة"""
        # 1. تحميل البيانات
        data = pd.read_csv(data_path)

        # 2. تحضير الميزات
        features_list = []
        labels = []

        for idx, row in data.iterrows():
            url = row['url']

            # استخراج الميزات
            features = self.prepare_reserved_ip_features(url)
            features_list.append(features)

            # استخراج التسمية
            if 'label' in row:
                label = row['label']
            elif 'result' in row:
                label = row['result']
            elif 'type' in row:
                label = 1 if str(row['type']).lower() in ['phishing', 'malicious', '1'] else 0
            else:
                label = 0

            labels.append(label)

        # 3. تحويل إلى DataFrame
        X = pd.DataFrame(features_list)
        self.feature_names = X.columns.tolist()

        # تحويل التسميات
        y = np.array([1 if str(l).lower() in ['phishing', 'malicious', '1', 'true'] else 0 for l in labels])
        print(f"📊 توزيع التسميات: {sum(y)} تصيد, {len(y)-sum(y)} آمن")

        # 4. تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )

        # 5. تطبيع البيانات
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # 6. تدريب نموذج XGBoost
        print("\nجاري تدريب نموذج XGBoost...")
        start_time = time.time()

        self.xgb_model = xgb.XGBClassifier(
            n_estimators=150,
            max_depth=7,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            eval_metric='logloss'
        )

        self.xgb_model.fit(
            X_train_scaled, y_train,
            eval_set=[(X_test_scaled, y_test)],
            verbose=False
        )

        train_time = time.time() - start_time

        # 7. تقييم النموذج
        y_pred = self.xgb_model.predict(X_test_scaled)
        y_proba = self.xgb_model.predict_proba(X_test_scaled)[:, 1]

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        auc_roc = roc_auc_score(y_test, y_proba)

        self.model_performance = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc_roc': auc_roc,
            'train_time': train_time,
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'feature_count': X.shape[1]
        }

        print(f"\n✅تم تدريب النموذج بنجاح!")
        print(f"📊أداء النموذج:")
        print(f"•accuracy: {accuracy:.4f}")
        print(f"•precision: {precision:.4f}")
        print(f"•recall: {recall:.4f}")
        print(f"•F1: {f1:.4f}")
        print(f"•AUC-ROC: {auc_roc:.4f}")
        print(f"•TRAIN-TIME: {train_time:.2f} ")

        self.is_trained = True

        return self.model_performance

    def predict_single_url(self, url):
        """التنبؤ برابط واحد"""
        if not self.is_trained:
            raise ValueError("النموذج غير مدرب. الرجاء التدريب أولاً.")

        # استخراج الميزات
        features = self.prepare_reserved_ip_features(url)
        features_df = pd.DataFrame([features])

        # ملء القيم الناقصة
        features_df = features_df.fillna(0)

        # التأكد من وجود جميع الميزات
        for col in self.scaler.feature_names_in_:
            if col not in features_df.columns:
                features_df[col] = 0

        # إعادة ترتيب الأعمدة
        features_df = features_df[self.scaler.feature_names_in_]

        # تطبيع
        features_scaled = self.scaler.transform(features_df)

        # التنبؤ
        prediction = self.xgb_model.predict(features_scaled)[0]
        proba = self.xgb_model.predict_proba(features_scaled)[0]
        confidence = proba[1] if prediction == 1 else proba[0]

        result = 'PHISHING' if prediction == 1 else 'BENIGN'

        # تفسير النتيجة
        explanation = self._explain_prediction(features, prediction)

        return {
            'url': url,
            'prediction': result,
            'confidence': confidence,
            'model': 'XGBoost (مدرب على IPs محجوزة)',
            'explanation': explanation,
            'features_used': len(features),
            'proba_phishing': proba[1],
            'proba_benign': proba[0]
        }

    def _explain_prediction(self, features, prediction):
        """تفسير سبب التنبؤ"""
        reasons = []

        if prediction == 1:  # PHISHING
            if features.get('has_admin_page', 0) == 1:
                reasons.append("صفحة إدارة على IP محجوز")

            if features.get('dangerous_kw_count', 0) >= 2:
                reasons.append(f"عدد الكلمات الخطيرة: {features.get('dangerous_kw_count', 0)}")

            if features.get('has_https', 0) == 0:
                reasons.append("لا يستخدم HTTPS")

            if features.get('has_risky_params', 0) == 1:
                reasons.append("معلمات خطرة")

            if features.get('is_private', 0) == 1:
                reasons.append("IP خاص")

            if features.get('suspicious_chars', 0) > 0:
                reasons.append("رموز مشبوهة")

        else:  # BENIGN
            if features.get('has_https', 0) == 1:
                reasons.append("يستخدم HTTPS")

            if features.get('dangerous_kw_count', 0) == 0:
                reasons.append("لا توجد كلمات خطيرة")

            if features.get('has_admin_page', 0) == 0:
                reasons.append("ليست صفحة إدارة")

            if features.get('has_risky_params', 0) == 0:
                reasons.append("لا توجد معلمات خطرة")

        return reasons if reasons else ["بناءً على تحليل الميزات"]


# ================ نظام المقارنة بين XGBoost والخوارزمية ================

class XGBoostVsAlgorithmComparator:
    """مقارنة بين نموذج XGBoost والخوارزمية القاعدية"""

    def __init__(self):
        self.xgb_model = ReservedIPsXGBoostModel()
        self.algorithm_detector = ReservedIPPhishingDetector()
        self.comparison_results = []

    def run_comparison(self, data_path, num_samples=10):
        # 1. تحميل البيانات
        data = pd.read_csv(data_path)

        # أخذ أول num_samples عينات
        if len(data) > num_samples:
            sample_data = data.head(num_samples).copy()
        else:
            sample_data = data.copy()

        # 2. تدريب نموذج XGBoost إذا لم يكن مدرباً
        if not self.xgb_model.is_trained:
            self.xgb_model.train_on_reserved_data(data_path)

        # 3. تطبيق المقارنة على كل رابط
        print(f"\n🔍 جاري تطبيق المقارنة...")

        comparison_data = []

        for idx, row in sample_data.iterrows():
            url = row['url']

            print(f"\n[{idx+1}/{len(sample_data)}] تحليل: {url[:60]}...")

            # الحصول على التسمية الحقيقية إذا وجدت
            true_label = None
            if 'label' in row:
                true_label = row['label']
            elif 'result' in row:
                true_label = row['result']

            # تحويل التسمية الحقيقية إلى نصي
            if true_label is not None:
                true_label_str = 'PHISHING' if str(true_label).lower() in ['phishing', 'malicious', '1', 'true'] else 'BENIGN'
            else:
                true_label_str = 'UNKNOWN'

            # التنبؤ بواسطة XGBoost
            print("🤖 XGBoost: جاري التنبؤ...")
            try:
                xgb_result = self.xgb_model.predict_single_url(url)
            except Exception as e:
                print(f"❌ خطأ في XGBoost: {e}")
                xgb_result = {'prediction': 'ERROR', 'confidence': 0.0}

            # التنبؤ بواسطة الخوارزمية
            print("⚡ الخوارزمية: جاري التحليل...")
            try:
                algo_result = self.algorithm_detector.detect_phishing_by_rules_enhanced_v2(url)
            except Exception as e:
                print(f"❌ خطأ في الخوارزمية: {e}")
                algo_result = {'prediction': 'ERROR', 'confidence': 0.0}

            # مقارنة النتائج
            agreement = (xgb_result['prediction'] == algo_result['prediction'])

            # مقارنة مع التسمية الحقيقية إذا كانت متوفرة
            if true_label_str != 'UNKNOWN':
                xgb_correct = (xgb_result['prediction'] == true_label_str)
                algo_correct = (algo_result['prediction'] == true_label_str)
            else:
                xgb_correct = None
                algo_correct = None

            # تجميع النتائج
            comparison_entry = {
                'index': idx + 1,
                'url': url,
                'true_label': true_label_str,

                # نتائج XGBoost
                'xgb_prediction': xgb_result['prediction'],
                'xgb_confidence': xgb_result.get('confidence', 0.0),
                'xgb_proba_phishing': xgb_result.get('proba_phishing', 0.0),
                'xgb_proba_benign': xgb_result.get('proba_benign', 0.0),
                'xgb_explanation': xgb_result.get('explanation', []),
                'xgb_correct': xgb_correct,

                # نتائج الخوارزمية
                'algo_prediction': algo_result.get('prediction', 'ERROR'),
                'algo_confidence': algo_result.get('confidence', 0.0),
                'algo_score': algo_result.get('score', 0.0),
                'algo_severity': algo_result.get('severity', 'UNKNOWN'),
                'algo_reasons': algo_result.get('reasons', []),
                'algo_correct': algo_correct,

                # مقارنة
                'agreement': agreement,
                'disagreement': not agreement,
                'winner': self._determine_winner(xgb_result, algo_result, true_label_str)
            }

            comparison_data.append(comparison_entry)

        self._statistical_analysis(comparison_data)

        # 5. حفظ النتائج
        self.comparison_results = comparison_data
        self._save_comparison_results(comparison_data)

        return comparison_data

    def _determine_winner(self, xgb_result, algo_result, true_label):
        """تحديد الفائز في حالة الاختلاف"""
        if true_label == 'UNKNOWN':
            return 'UNKNOWN'

        xgb_correct = (xgb_result['prediction'] == true_label)
        algo_correct = (algo_result['prediction'] == true_label)

        if xgb_correct and not algo_correct:
            return 'XGBoost'
        elif algo_correct and not xgb_correct:
            return 'Algorithm'
        elif xgb_correct and algo_correct:
            return 'Both Correct'
        else:
            return 'Both Wrong'

    def _statistical_analysis(self, comparison_data):
        """تحليل إحصائي للنتائج"""
        total = len(comparison_data)
        # 1. حساب الاتفاق
        agreements = sum(1 for entry in comparison_data if entry['agreement'])
        disagreements = total - agreements
        agreement_rate = agreements / total if total > 0 else 0
        print(f"\n📊 إحصائيات الاتفاق:")
        print(f"• إجمالي العينات: {total}")
        print(f"• حالات الاتفاق: {agreements} ({agreement_rate:.1%})")
        print(f"• حالات الاختلاف: {disagreements} ({disagreements/total:.1%})")

        # 2. حساب الدقة (إذا كانت التسميات الحقيقية متوفرة)
        entries_with_true_label = [e for e in comparison_data if e['true_label'] != 'UNKNOWN']

        if entries_with_true_label:
            true_label_count = len(entries_with_true_label)

            xgb_correct = sum(1 for e in entries_with_true_label if e['xgb_correct'])
            algo_correct = sum(1 for e in entries_with_true_label if e['algo_correct'])

            xgb_accuracy = xgb_correct / true_label_count if true_label_count > 0 else 0
            algo_accuracy = algo_correct / true_label_count if true_label_count > 0 else 0

            print(f"\n🎯 الدقة :")
            print(f"• XGBoost: {xgb_accuracy:.1%} ({xgb_correct}/{true_label_count})")
            print(f"• الخوارزمية: {algo_accuracy:.1%} ({algo_correct}/{true_label_count})")

            # تحديد الفائز
            if xgb_accuracy > algo_accuracy:
                winner = "XGBoost"
                diff = xgb_accuracy - algo_accuracy
            elif algo_accuracy > xgb_accuracy:
                winner = "الخوارزمية"
                diff = algo_accuracy - xgb_accuracy
            else:
                winner = "تعادل"
                diff = 0

            print(f"🏆 الفائز: {winner} (فرق: {diff:.1%})")

        # 3. تحليل الثقة
        xgb_confidences = [e['xgb_confidence'] for e in comparison_data if e['xgb_confidence'] is not None]
        algo_confidences = [e['algo_confidence'] for e in comparison_data if e['algo_confidence'] is not None]

        if xgb_confidences and algo_confidences:
            avg_xgb_conf = np.mean(xgb_confidences)
            avg_algo_conf = np.mean(algo_confidences)

            print(f"\n📈 متوسط الثقة:")
            print(f"• XGBoost: {avg_xgb_conf:.1%}")
            print(f"• الخوارزمية: {avg_algo_conf:.1%}")

        # 4. تحليل النتائج حسب النوع
        print(f"\n📋 توزيع النتائج:")
        xgb_counts = {}
        algo_counts = {}

        for entry in comparison_data:
            xgb_pred = entry['xgb_prediction']
            algo_pred = entry['algo_prediction']

            xgb_counts[xgb_pred] = xgb_counts.get(xgb_pred, 0) + 1
            algo_counts[algo_pred] = algo_counts.get(algo_pred, 0) + 1

        print(f"• XGBoost: {xgb_counts}")
        print(f"• الخوارزمية: {algo_counts}")

    def _save_comparison_results(self, comparison_data):
        """حفظ نتائج المقارنة"""
        # تحويل إلى DataFrame
        results_df = pd.DataFrame(comparison_data)

        # حفظ بتنسيق CSV
        timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"/content/xgboost_vs_algorithm_comparison_{timestamp}.csv"

        return output_file

def run_xgboost_vs_algorithm_comparison():
    """تشغيل مقارنة XGBoost vs الخوارزمية"""

    # التحقق من وجود ملف البيانات
    import os
    if not os.path.exists(RESERVED_ONLY_DATA_FILE):
        print(f"❌ ملف الداتا غير موجود: {RESERVED_ONLY_DATA_FILE}")
        return None

    # إنشاء نظام المقارنة
    comparator = XGBoostVsAlgorithmComparator()

    try:
        # تشغيل المقارنة على أول 10 عينات
        comparison_results = comparator.run_comparison(RESERVED_ONLY_DATA_FILE, num_samples=10)

        # عرض تقرير تفصيلي
        print("\n" + "="*100)
        print("📋 تقرير مفصل للمقارنة")
        print("="*100)

        for entry in comparison_results:
            print(f"\n[{entry['index']}] {entry['url'][:70]}...")
            print(f"🤖 XGBoost: {entry['xgb_prediction']} (ثقة: {entry['xgb_confidence']:.1%})")
            print(f"💡 تفسير: {', '.join(entry['xgb_explanation'][:2]) if entry['xgb_explanation'] else 'لا يوجد'}")
            print(f"⚡ الخوارزمية: {entry['algo_prediction']} (ثقة: {entry['algo_confidence']:.1%})")
            print(f"📊 نقاط: {entry.get('algo_score', 0):.2f} | شدة: {entry.get('algo_severity', 'UNKNOWN')}")
            print(f"💡 أسباب: {', '.join(entry['algo_reasons'][:2]) if entry['algo_reasons'] else 'لا يوجد'}")
            if entry['true_label'] != 'UNKNOWN':
                print(f"🎯 التسمية الحقيقية: {entry['true_label']}")
                print(f"✅ XGBoost صحيح: {'نعم' if entry['xgb_correct'] else 'لا'}")
                print(f"✅ الخوارزمية صحيحة: {'نعم' if entry['algo_correct'] else 'لا'}")
            print(f"⚖️الاتفاق: {'نعم' if entry['agreement'] else 'لا'}")
            if not entry['agreement']:
                print(f"🏆 الفائز: {entry['winner']}")

        return comparison_results

    except Exception as e:
        print(f"❌ حدث خطأ في المقارنة: {e}")
        import traceback
        traceback.print_exc()
        return None
   # ================ التشغيل الرئيسي ================

if __name__ == "__main__":
    # إعدادات العرض
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.precision', 10)
    print("\n"+"="*100)
    print("🚀 بدء تشغيل نظام الكشف عن التصيد")
    print("="*100)

    # ============ المرحلة 1: النظام الكامل على الداتا الأولى ============
    print("\n"+"="*100)
    print("📊 المرحلة 1: تشغيل النظام الكامل على الداتا الأولى")

    # ثانياً: تشغيل النظام الكامل على الداتا الأولى
    system = CompletePhishingDetectionSystem()

    try:
        # التحقق من وجود الملف الرئيسي
        import os

        # تحميل وتقسيم البيانات أولاً
        data_split = system.load_and_split_data(MAIN_DATA_FILE)

        if len(data_split['all_data']) == 0:
            print("❌ فشل في تحميل البيانات!")
            print("   • تأكد من وضع ملف balanced_urls.csv في /content/")
            raise ValueError("البيانات فارغة")

        # تدريب النظام الكامل
        training_results = system.train_system(MAIN_DATA_FILE)

        # تحليل وتقييم IPs المحجوزة
        reserved_analysis_results = system.analyze_and_test_reserved_ips(data_split)

        # ============ المرحلة 2: الخوارزمية فقط على الداتا الثانية ============
        print("\n"+"="*100)
        print("🔧 المرحلة 2: تشغيل الخوارزمية فقط على الداتا الجديدة")

        # التحقق إذا كان الملف موجود قبل التشغيل
        if os.path.exists(RESERVED_ONLY_DATA_FILE):
            print(f"✅ تم العثور على ملف الداتا الثانية: {RESERVED_ONLY_DATA_FILE}")
            reserved_results = run_reserved_detector_silent()
            if reserved_results is not None:
                print(f"✅ تم تحليل جميع الروابط من الداتا الثانية")
        else:
            print(f"⚠️  ملف الداتا الثانية غير موجود: {RESERVED_ONLY_DATA_FILE}")
            print("   • تخطي المرحلة 2")
            reserved_results = None

        # ============ المرحلة 3: مقارنة الخوارزمية القاعدية مع النموذج المدرّب ============
        print("\n"+"="*100)
        print("🤖 المرحلة 3: مقارنة الخوارزمية القاعدية مع النموذج المدرّب")

        # المقارنة
        if os.path.exists(RESERVED_ONLY_DATA_FILE):
            comparison_results = run_xgboost_vs_algorithm_comparison()
            if comparison_results:
                print("\n")
                print(f"✅ تمت مقارنة {len(comparison_results)} عينة بنجاح")
        else:
            print(f"⚠️  ملف الداتا الثانية غير موجود: {RESERVED_ONLY_DATA_FILE}")
            comparison_results = None

        # ============ المرحلة 5: الواجهة التفاعلية ============
        print("\n" + "="*100)
        while True:
            print("\n" + "="*60)
            print("📋 خيارات إضافية:")
            print("="*60)
            print("1. 🔍 تحليل رابط معين بالنظام الكامل")
            print("2. 🤖 تحليل رابط معين بـ XGBoost فقط")
            print("3. ⚡ تحليل رابط معين بالخوارزمية فقط")
            print("4. 📊 إعادة تشغيل المقارنة")
            print("5. 📈 عرض أداء XGBoost")
            print("6. 📊 عرض أداء الخوارزمية")
            print("7. 🚪 الخروج")

            choice = input("\n🎯 اختر خياراً (1-8): ").strip()

            if choice == '1':
                url = input("\n🔗 أدخل الرابط: ").strip()
                if url:
                    print(f"\n📡 جاري تحليل الرابط: {url[:80]}...")
                    result = system.predict_url(url)
                    print(f"\n✅ النتيجة النهائية:")
                    for key, value in result.items():
                        print(f"   • {key}: {value}")
                else:
                    print("❌ لم تدخل رابطاً!")

            elif choice == '2':
                url = input("\n🔗 أدخل الرابط: ").strip()
                if url:
                    try:
                        # التحقق إذا كان النموذج مدرباً
                        xgb_model = ReservedIPsXGBoostModel()
                        if not xgb_model.is_trained and os.path.exists(RESERVED_ONLY_DATA_FILE):
                            print("⚙️ جاري تدريب نموذج XGBoost أولاً...")
                            xgb_model.train_on_reserved_data(RESERVED_ONLY_DATA_FILE)

                        result = xgb_model.predict_single_url(url)
                        print(f"\n🤖 نتيجة XGBoost:")
                        print(f"   📍 الرابط: {result['url'][:80]}...")
                        print(f"   🎯 النتيجة: {result['prediction']}")
                        print(f"   📊 الثقة: {result['confidence']:.2%}")
                        print(f"   ⚠️  احتمال التصيد: {result['proba_phishing']:.2%}")
                        print(f"   💡 التفسير: {', '.join(result['explanation'][:3])}")
                    except Exception as e:
                        print(f"❌ خطأ: {e}")
                else:
                    print("❌ لم تدخل رابطاً!")

            elif choice == '3':
                url = input("\n🔗 أدخل الرابط: ").strip()
                if url:
                    detector = ReservedIPPhishingDetector()
                    result = detector.detect_phishing_by_rules_enhanced_v2(url)
                    print(f"\n⚡ نتيجة الخوارزمية:")
                    print(f"   📍 الرابط: {url[:80]}...")
                    print(f"   🎯 النتيجة: {result['prediction']}")
                    print(f"   📊 الثقة: {result['confidence']:.2%}")
                    print(f"   ⚖️ النقاط: {result.get('score', 0):.2f}")
                    print(f"   ⚠️  الشدة: {result.get('severity', 'UNKNOWN')}")
                    if 'reasons' in result:
                        print(f"   💡 الأسباب: {', '.join(result['reasons'][:3])}")
                else:
                    print("❌ لم تدخل رابطاً!")

            elif choice == '4':
                if os.path.exists(RESERVED_ONLY_DATA_FILE):
                    print("🔄 جاري إعادة تشغيل المقارنة...")
                    comparison_results = run_xgboost_vs_algorithm_comparison()
                    if comparison_results:
                        print(f"✅ تمت مقارنة {len(comparison_results)} عينة بنجاح")
                else:
                    print(f"❌ الملف غير موجود: {RESERVED_ONLY_DATA_FILE}")

            elif choice == '5':
                if 'comparison_results' in locals() and comparison_results:
                    xgb_correct = sum(1 for e in comparison_results if e.get('xgb_correct'))
                    total = len([e for e in comparison_results if e.get('true_label') != 'UNKNOWN'])
                    if total > 0:
                        accuracy = xgb_correct / total
                        print(f"\n📊 أداء XGBoost في المقارنة:")
                        print(f"   🎯 الدقة: {accuracy:.1%} ({xgb_correct}/{total})")
                        print(f"   📊 متوسط الثقة: {np.mean([e['xgb_confidence'] for e in comparison_results]):.1%}")
                    else:
                        print("❌ لا توجد بيانات كافية لتقييم أداء XGBoost")
                else:
                    print("❌ لم يتم إجراء مقارنة بعد")

            elif choice == '6':
                if 'comparison_results' in locals() and comparison_results:
                    algo_correct = sum(1 for e in comparison_results if e.get('algo_correct'))
                    total = len([e for e in comparison_results if e.get('true_label') != 'UNKNOWN'])
                    if total > 0:
                        accuracy = algo_correct / total
                        print(f"\n📊 أداء الخوارزمية في المقارنة:")
                        print(f"   🎯 الدقة: {accuracy:.1%} ({algo_correct}/{total})")
                        print(f"   📊 متوسط الثقة: {np.mean([e['algo_confidence'] for e in comparison_results]):.1%}")
                    else:
                        print("❌ لا توجد بيانات كافية لتقييم أداء الخوارزمية")
                else:
                    print("❌ لم يتم إجراء مقارنة بعد")


            elif choice == '7':
                print("\n" + "="*60)
                print("👋 شكراً لاستخدام نظام الكشف عن التصيد!")
                print("="*60)
                break

            else:
                print("❌ خيار غير صالح، حاول مرة أخرى")

    except FileNotFoundError as e:
        print(f"❌ خطأ: {e}")

    except Exception as e:
        print(f"❌ حدث خطأ غير متوقع: {e}")
        import traceback
        traceback.print_exc()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.2 MB/s eta 0:00:00

🚀 بدء تشغيل نظام الكشف عن التصيد

📊 المرحلة 1: تشغيل النظام الكامل على الداتا الأولى
📂 جاري تحميل البيانات من: /content/balanced_urls.csv
✅ تم تحميل 632508 سجل
📊 أعمدة البيانات: ['url', 'label', 'result']
✅ الروابط العادية: 632505
✅ روابط IPs المحجوزة: 3
📂 جاري تحميل البيانات من: /content/balanced_urls.csv
✅ تم تحميل 632508 سجل
📊 أعمدة البيانات: ['url', 'label', 'result']
✅ الروابط العادية: 632505
✅ روابط IPs المحجوزة: 3
🔍 استخراج الميزات لـ 632505 رابط...
✅ تم تحضير 632505 سجل مع 33 ميزة
 شكل البيانات: (632505, 33)

 تدريب 7 نماذج متنوعة

 تدريب Random Forest...
📊 ACCURACY: 0.9818
🎯 F1: 0.9815
📈 AUC: 0.9950
⏱️ TIME/s: 44.33
🏆أصبح أفضل نموذج!

 تدريب XGBoost...
📊 ACCURACY: 0.9903
🎯 F1: 0.9902
📈 AUC: 0.9980
⏱️ TIME/s: 6.22
🏆أصبح أفضل نموذج!

 تدريب Log